In [ ]:
import folium
import pandas as pd
import gpxpy
from geopy.distance import geodesic
import matplotlib.pyplot as plt


map_center = (50.96326470728358, -0.9796756545673202) # day two campsite
m = folium.Map(location=[map_center[0], map_center[1]], zoom_start=11)

m

## Goal
- [x] Display the full route on a map with camping spots
- [ ] Determine speed
- [ ] display speed as a colour gradient on the map
- [ ] graph gradient vs speed

In [ ]:
gpx_days = []
for i in range(1,5):
    with open(f"activities/sdw_{i}.gpx", "r") as gpx_file:
        gpx_days.append(gpxpy.parse(gpx_file))


In [ ]:
colors = ["red", "blue", "green", "purple"]
for (day, color) in zip(gpx_days, colors): 
    for track in day.tracks:
        for segment in track.segments:
            points = [(point.latitude, point.longitude) for point in segment.points]
            folium.PolyLine(points, color=color, weight=2.5, opacity=0.5).add_to(m)

m

In [ ]:
gpx_days[0].tracks[0].segments[0].points[0]

In [ ]:
for day in gpx_days:
    print(len(day.tracks), len(day.tracks[0].segments), len(day.tracks[0].segments[0].points))

## Add speed to track object

In [ ]:
def calc_speed(point1, point2):
    """ Given two gpx points, return their speed in m/s"""
    distance = geodesic((point1.latitude, point1.longitude), (point2.latitude, point2.longitude)).meters
    time_diff = (point2.time - point1.time).total_seconds()
    return distance / time_diff


In [ ]:
for day in gpx_days:
    seg = day.tracks[0].segments[0]
    seg.points[0].speed = 0
    for i in range(1, len(seg.points)):
        seg.points[i].speed = calc_speed(seg.points[i-1], seg.points[i])

speeds = [p.speed for p in gpx_days[3].tracks[0].segments[0].points]


In [ ]:
plt.plot(speeds)
plt.xlabel('Data Point')
plt.ylabel('Speed')
plt.title('Speed Plot')
plt.show()


### Elevation change

In [ ]:
elevation_change = []
for day in gpx_days:
    elevs = [0]
    seg = day.tracks[0].segments[0]
    for i in range(1, len(seg.points)):
        elevs.append(seg.points[i].elevation - seg.points[i-1].elevation)
    elevation_change.append(elevs)

elevation_change

In [ ]:
plt.plot(elevation_change[3])
plt.xlabel('Data Point')
plt.ylabel('Elevation Change (m)')
plt.title('Elevation Change')
plt.show()


## Data Organising

Move the gpx points and elevation changes into a pandas dataframe

In [ ]:
temp = []

for (days, elevs) in zip(gpx_days,elevation_change):
    for (p, e) in zip(days.tracks[0].segments[0].points, elevs):
        temp.append({
            "latitude": p.latitude,
            "longitude": p.longitude,
            "time": p.time.timestamp(),
            "elevation_change": e,
            "elevation": p.elevation,
            "speed": p.speed * 3.6
        })

df = pd.DataFrame(temp)

In [ ]:
df.describe()

## Data Cleaning
* Gaussian smoothing on the speed data
* Z-score threshold, if above z score, default value is expected value

In [ ]:
from scipy.ndimage import gaussian_filter1d
import numpy as np 

speed = df['speed']
z_score_threshold = 3
z_scores = (speed - np.mean(speed)) / np.std(speed)

speed_mean = np.mean(speed)

speed_processed = [s if z < z_score_threshold else speed_mean for (s, z) in zip(speed, z_scores)]

sigma = 2  # You can adjust the sigma value to control the smoothing level
speed_smoothed = gaussian_filter1d(speed_processed, sigma=sigma)


df['speed_smoothed'] = speed_smoothed

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(df['time'], df['speed'], label='Original Speed')
plt.scatter(df['time'], df['speed_smoothed'], label='Smoothed Speed', color='red')
plt.xlabel('Time')
plt.ylabel('Speed')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

bins = 20
# plt.hist(df['speed'], bins=bins, alpha=0.5, label='Speed')
plt.hist(df['speed_smoothed'], bins=bins, alpha=0.5, label='Smoothed Speed')
plt.xlabel('Speed')
plt.ylabel('Frequency')
plt.title('Speed vs Smoothed Speed Histogram')
plt.legend()
plt.show()


In [ ]:
output = df.copy()
output['speed'] = output['speed_smoothed']
output.drop('speed_smoothed', axis=1, inplace=True)

output.to_csv('south_downs_way.csv')

## Advanced visualisations

Overlay speed variable as color over total route graph

In [ ]:
from folium import plugins
import branca.colormap as cm

def create_variate_map(df, column_name, units, save=False):

    m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=11)

    upper_column_name = column_name[0].upper() + column_name[1:]
    # Create a color scale
    colormap = cm.LinearColormap(colors=['green', 'yellow', 'red'], vmin=df[column_name].min(), vmax=df[column_name].max())
    colormap.caption = f'{upper_column_name} ({units})'

    # Add points to the map
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=2,
            popup=f"{upper_column_name}: {row[column_name]} {units}",
            color=colormap(row[column_name]),
            fill=True,
            fill_color=colormap(row[column_name]),
            fill_opacity=0.7
        ).add_to(m)

    # Add color scale to map
    m.add_child(colormap)

    # Save map to an HTML file
    if save:
        m.save('map.html')
    return m


In [ ]:
df['latitude'].mean(), df['longitude'].mean()

In [ ]:
speed_map = create_variate_map(df, "speed", "km/h")
speed_map

In [ ]:
elevation_map = create_variate_map(df, "elevation", "m")
elevation_map

In [ ]:
smoothed_speed_map = create_variate_map(df, "speed_smoothed", "km/h")
smoothed_speed_map

## Switching between layers

In [ ]:
def create_feature_group(data, column_name,units,map, colors=['green', 'yellow', 'red']):
    upper_column_name = column_name[0].upper() + column_name[1:]

    colormap = cm.LinearColormap(colors, vmin=df[column_name].min(), vmax=df[column_name].max())
    colormap.caption = f'{upper_column_name} ({units})'

    feature_group = folium.FeatureGroup(name=upper_column_name)
    for _, row in data.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=2,
            popup=f"{upper_column_name}: {row[column_name]} {units}",
            color=colormap(row[column_name]),
            fill=True,
            fill_color=colormap(row[column_name]),
            fill_opacity=0.7
        ).add_to(feature_group)
    map.add_child(colormap)
    return feature_group

In [ ]:


layer_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=11)

# Create feature groups for each dataset
feature_group1 = create_feature_group(df, 'speed', 'm/s', layer_map)
feature_group2 = create_feature_group(df, 'elevation', 'm', layer_map)

# Add feature groups to the map
feature_group1.add_to(layer_map)
feature_group2.add_to(layer_map)

# Add LayerControl to switch between layers
folium.LayerControl().add_to(layer_map)

# layer_map.save('layermap.html')

layer_map